In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
import sklearn
from ctypes import *
WinDLL('G:\\CUDA\\cudnn\\cuda\\bin\\cudnn_ops_infer64_8.dll')
WinDLL('G:\\CUDA\\cudnn\\cuda\\bin\\cudnn_adv_infer64_8.dll')
WinDLL('G:\\CUDA\\cudnn\\cuda\\bin\\cudnn_adv_train64_8.dll')

<WinDLL 'G:\CUDA\cudnn\cuda\bin\cudnn_adv_train64_8.dll', handle 7ff81cca0000 at 0x18b94f8ae50>

In [4]:
eeg_df = pd.read_excel('eegdata.xlsx')
ques_df = pd.read_excel('questionare.xlsx')
ques_df = ques_df.loc[:,['sub','group','gender','age','外向性','宜人性','尽责性','神经质','开放性']]

In [5]:
df_initial = pd.merge(eeg_df,ques_df,on=['sub'])
#将分类变量转为0与1
df_initial.replace('male',1,inplace=True)
df_initial.replace('female',0,inplace=True)
df_initial['group'].replace(1,0,inplace=True)  #group为0的是被社会拒斥的        ##！！！！group实际上不应该出现在我的实验中，考虑删掉这个变量
df_initial['group'].replace(2,1,inplace=True)  #group为1的是未被社会拒斥的
df_initial['con'].replace('neg',0,inplace=True)
df_initial['con'].replace('pos',1,inplace=True)


df_initial.drop(columns=['sub','group'],inplace=True)
x = df_initial.drop(columns=['外向性','宜人性','尽责性','神经质','开放性'])
x = pd.DataFrame(StandardScaler().fit_transform(x),columns=x.columns)

y = df_initial.loc[:,['外向性','宜人性','尽责性','神经质','开放性']]

In [6]:
import eli5
from eli5.sklearn import PermutationImportance
i='外向性'
y_temp = y.loc[:,[i]]
x_train,x_test,y_train,y_test = train_test_split(x,y_temp,test_size=0.15,random_state=1)
model = keras.models.load_model('models\\'+i+'.h5')
perm = PermutationImportance(model, scoring='neg_mean_squared_error',random_state=1).fit(x_test, y_test)
eli5.show_weights(perm, feature_names = x_test.columns.tolist())


KeyboardInterrupt: 